In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
import statsmodels.api as sm
import pandas as pd
import numpy as np
import re

from bag_of_words import *
from regex_predictor import *
from odds_ratio import *
from proximal import *

In [2]:
master_data = pd.read_csv('csv_files/master_data.csv')

### Bag of words linear regression to see if the text data carries any predictive information on heart failure.

In [3]:
text_clf = create_bow_model(max_words=10000, ngram_range=(1, 2))

text_clf.fit(master_data['notes_half2'], master_data['heart_fail'])

/home/jchen/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('vect',
                 CountVectorizer(max_features=10000, ngram_range=(1, 2))),
                ('tfidf', TfidfTransformer()),
                ('lr', LogisticRegression(C=1e+20))])

In [6]:
print('majority class', 1-np.mean(master_data['heart_fail']))

majority class 0.7359003089878102


In [4]:
predicted = text_clf.predict(master_data['notes_half2'])

print(np.mean(predicted == master_data['heart_fail']))

0.8635360429187464


In [5]:
create_confusion_matrix(master_data['heart_fail'], predicted)

{'tn': 20237,
 'fp': 1436,
 'fn': 2583,
 'tp': 5195,
 'sensitivity': 0.6679094883003343,
 'specificity': 0.9337424445162184,
 'precision': 0.7834414115518021,
 'recall': 0.6679094883003343}

Find what words are used most often.

In [9]:
coef = text_clf['lr'].coef_[0]
print(coef)

words = text_clf['vect'].get_feature_names_out()
print(words)

sorted({w:c for w, c in zip(words, coef)}.items(), key=lambda kv: -kv[1])[4000:5000]

[-0.32397592  2.08844662  1.10139299 ...  4.89953384 -0.96869696
  2.41889985]
['00' '02' '0cm2' ... 'ystolic' 'zofran' 'zosyn']


[('healing', 0.6716482481935407),
 ('tachycardia', 0.6684002844934771),
 ('are filamentous', 0.6676183684574346),
 ('follows', 0.6673852134791473),
 ('report images', 0.6665968304768569),
 ('nondistended', 0.6662049569434116),
 ('hypokinesis right', 0.6657199514972943),
 ('study may', 0.6653238573211637),
 ('usion', 0.6652124798626199),
 ('leaflet function', 0.6648955782416314),
 ('strength', 0.6648222502147186),
 ('dressing intact', 0.6638279856441689),
 ('coanda', 0.6630189970542599),
 ('to hyperdynamic', 0.6625454575227055),
 ('no residual', 0.6620583457336978),
 ('much', 0.6597064055067818),
 ('underestimated due', 0.6589077808049433),
 ('the rest', 0.6576355959694543),
 ('nurse', 0.6575662799412874),
 ('is inadequate', 0.6572832328686552),
 ('quality severe', 0.6568787393651386),
 ('atherosclerotic', 0.6566477305947139),
 ('defibrillator pads', 0.6563413033144149),
 ('central', 0.6560336938728707),
 ('mr aorta', 0.6555320089404241),
 ('2104', 0.6531461514300536),
 ('us', 0.6528592

In [6]:
# atrial, ventricle, cardio, **valve
regex_predicted = regular_expression_predict(master_data['notes_half2'], ['valve'])

print(np.mean(regex_predicted))

print(np.mean(regex_predicted == master_data['heart_fail']))

create_confusion_matrix(master_data['heart_fail'], regex_predicted)

0.5566194696275168
0.5984856201826763


{'tn': 11453,
 'fp': 10220,
 'fn': 1605,
 'tp': 6173,
 'sensitivity': 0.7936487528927745,
 'specificity': 0.5284455313062336,
 'precision': 0.3765631672055145,
 'recall': 0.7936487528927745}